New approach entirely

In [78]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [79]:


# Constants
IMG_SIZE = 224
BATCH_SIZE = 29

# Data paths
TRAIN_DIR = "../Data/train"
VALID_DIR = "../Data/valid"
TEST_DIR  = "../Data/test"



def load_dataset_from_csv(folder_path):
    df = pd.read_csv(os.path.join(folder_path, "labels.csv"))
    image_paths = [os.path.join(folder_path, fname) for fname in df['filename']]
    labels = df['fallen'].values

    def process_image(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE)
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    path_ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    image_label_ds = path_ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    return image_label_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)



In [80]:
train_ds = load_dataset_from_csv(TRAIN_DIR)
valid_ds = load_dataset_from_csv(VALID_DIR)
test_ds  = load_dataset_from_csv(TEST_DIR)


In [81]:
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary output
])


In [82]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=10)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 358ms/step - accuracy: 0.5772 - loss: 0.7979 - val_accuracy: 0.7966 - val_loss: 0.5110
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 349ms/step - accuracy: 0.7736 - loss: 0.4744 - val_accuracy: 0.7966 - val_loss: 0.6142
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 354ms/step - accuracy: 0.7970 - loss: 0.4608 - val_accuracy: 0.7966 - val_loss: 0.4837
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 353ms/step - accuracy: 0.8099 - loss: 0.4357 - val_accuracy: 0.7966 - val_loss: 0.5296
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 359ms/step - accuracy: 0.8339 - loss: 0.4107 - val_accuracy: 0.7966 - val_loss: 0.5334
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 368ms/step - accuracy: 0.8380 - loss: 0.3722 - val_accuracy: 0.7966 - val_loss: 0.5218
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 368ms/step - accuracy: 0.8731 - loss: 0.3568 - val_accuracy: 0.8136 - val_loss: 0.5076
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 371ms/step - accuracy: 0.8794 - loss: 0.3067 - val_accuracy: 0.

In [83]:
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc:.2%}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8888 - loss: 0.2964
Test Accuracy: 89.83%


In [84]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),  # 👈 input shape goes here
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=10)


Epoch 1/10


/Users/drapcat/4380env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 393ms/step - accuracy: 0.6661 - loss: 0.9502 - val_accuracy: 0.6441 - val_loss: 0.6224
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 391ms/step - accuracy: 0.6829 - loss: 0.5916 - val_accuracy: 0.7627 - val_loss: 0.7115
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 400ms/step - accuracy: 0.7486 - loss: 0.4980 - val_accuracy: 0.7966 - val_loss: 0.7917
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 393ms/step - accuracy: 0.7968 - loss: 0.4609 - val_accuracy: 0.8136 - val_loss: 0.5625
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 394ms/step - accuracy: 0.8036 - loss: 0.4690 - val_accuracy: 0.7966 - val_loss: 0.6506
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 396ms/step - accuracy: 0.7896 - loss: 0.4200 - val_accuracy: 0.8136 - val_loss: 0.5141
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 395ms/step - accuracy: 0.8132 - loss: 0.4134 - val_accuracy: 0.7966 - val_loss: 0.6613
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 406ms/step - accuracy: 0.8149 - loss: 0.4535 - val_accuracy: 0.8136 - val_

In [85]:
def load_dataset_with_filenames(folder_path):
    df = pd.read_csv(os.path.join(folder_path, "labels.csv"))
    image_paths = [os.path.join(folder_path, fname) for fname in df['filename']]
    labels = df['fallen'].values

    def process(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE)
        image = tf.cast(image, tf.float32) / 255.0
        return image, label, path  # keep path for reference

    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    return ds.map(process).batch(BATCH_SIZE)


predictions = []
for images, labels, paths in load_dataset_with_filenames(TEST_DIR):
    probs = model.predict(images).flatten()
    preds = (probs > 0.5).astype(int)
    for i in range(len(images)):
        predictions.append({
            "filename": paths[i].numpy().decode(),
            "predicted": int(preds[i]),
            "actual": int(labels[i].numpy()),
            "confidence": float(probs[i])
        })


import pandas as pd
results_df = pd.DataFrame(predictions)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


2025-07-21 16:19:52.643829: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [86]:
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc:.2%}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8246 - loss: 0.3370
Test Accuracy: 84.75%


In [87]:
# See mismatches
errors = results_df[results_df["predicted"] != results_df["actual"]]

# Preview a few
errors.head(20)


,filename,predicted,actual,confidence
1,../Data/test/IMG_0449_jpeg_jpg.rf.64e93f0e7001...,0,1,0.194553
2,../Data/test/IMG_0433_jpeg_jpg.rf.1485f0e51d7c...,0,1,0.359566
4,../Data/test/IMG_0458_jpeg_jpg.rf.2c5e4a9004b3...,0,1,0.457838
8,../Data/test/IMG_20210928_102051_jpg.rf.3df82b...,1,0,0.861663
10,../Data/test/IMG_20210928_102203_jpg.rf.f22349...,1,0,0.909025
12,../Data/test/IMG_0511_jpeg_jpg.rf.b444f2176b30...,0,1,0.324991
16,../Data/test/IMG_0505_jpeg_jpg.rf.9661dc857dab...,0,1,0.402342
36,../Data/test/20210928_164146_jpg.rf.8527ef8099...,0,1,0.005947
48,../Data/test/IMG_0428_jpeg_jpg.rf.74a4b207cf12...,0,1,0.198675


In [88]:
from sklearn.metrics import confusion_matrix, classification_report


In [89]:
y_true = []
y_pred = []

for images, labels in test_ds:
    probs = model.predict(images).flatten()
    preds = (probs > 0.5).astype(int)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# Display confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Not Fallen", "Fallen"]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Confusion Matrix:
[[42  2]
 [ 7  8]]

Classification Report:
              precision    recall  f1-score   support

  Not Fallen       0.86      0.95      0.90        44
      Fallen       0.80      0.53      0.64        15

    accuracy                           0.85        59
   macro avg       0.83      0.74      0.77        59
weighted avg       0.84      0.85      0.84        59

